<a href="https://colab.research.google.com/github/swissecon-cryptecon/public_blockchains/blob/main/Public_Blockchains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Applications on Public Blockchains



Content

1.   Chapter 1: Connection to Messari and functions to simplify download
2.   Chapter 2: Sleect and downlaod data (for different assets, different variables)
3. Chapter 3: Visualizations


Note: List of variable available as a CSV, see https://github.com/swissecon-cryptecon/public_blockchains/blob/main/messari_metrics.csv

## 1. Connection to Messari and functions to simplify download



In [ ]:
#!pip install messari

In [13]:
from messari.messari import Messari
import pandas as pd
import plotly.express as px

In [14]:
# Messari API Key
messari = Messari('OwwZ+mQ92hTCnYuzB-Gx9+927B2h-PNBIiyihjsVMmnF3to4')

In [15]:
#list of variables, see also documentation: https://messari.io/api/docs#tag/News/operation/Get%20News%20for%20Asset
#print(dir(messari)) 
list_messari_metrics = messari.list_asset_timeseries_metric_ids()
data = list_messari_metrics["data"]["metrics"]  
df = pd.DataFrame(data)
df.to_csv("messari_metrics.csv", index=False)

In [16]:
asset = "solana"
metric = "act.addr.cnt"
start = '2017-01-01'
end = '2023-04-15'
interval = '1w'

query_params = {
    'start': start,
    'end': end,
    'interval': interval,
    'order': 'ascending',
    'format': 'json',
    'timestamp-format': 'rfc3339'
}
response = messari.get_asset_timeseries(asset_key=asset, metric_id=metric, **query_params)
response

{'status': {'elapsed': 15,
  'timestamp': '2023-04-16T16:47:14.556841755Z',
  'error_code': 404,
  'error_message': 'Not Found'}}

In [17]:
# Retrieve historical timeseries data for an asset.
def messari_query(asset, metric, start = '2017-01-01', end = '2023-04-15', interval = '1w'):

    query_params = {
        'start': start,
        'end': end,
        'interval': interval,
        'order': 'ascending',
        'format': 'json',
        'timestamp-format': 'rfc3339'
    }
    response = messari.get_asset_timeseries(asset_key=asset, metric_id=metric, **query_params)
    error_code = response["status"].get("error_code", None)
    error_msg = response['status'].get("error_message", None)

    if error_code is not None:
        error_msg = asset + ", " + metric + ": Error " +  error_msg
        print(error_msg)
        print("Metric '" + metric + "' for " + asset + " is ignored!")
        df = pd.DataFrame()
        df['timestamp'] = None
        df['asset'] = None
    else:
        # Assume the output dictionary is stored in a variable called 'output'
        data = response['data']
        values = data['values']

        # Convert the values list to a Pandas DataFrame
        df = pd.DataFrame(values, columns=data['schema']['values_schema'].keys())

        # Add a column for the asset name
        df['asset'] = asset

        # Convert the timestamp column to a datetime data type
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Print the resulting DataFrame
    return df

In [21]:
def multi_messari_query(assets, metrics, start = '2020-01-01', end = '2023-04-15', interval = '1w'):
    #initialize empty dataframe
    df_merged = pd.DataFrame()
    df_merged['timestamp'] = None
    df_merged['asset'] = None

    #loop through assets and metrics
    for asset in assets:
        df_merged_metric = pd.DataFrame()
        df_merged_metric['timestamp'] = None
        df_merged_metric['asset'] = None
        for metric in metrics:
            df = messari_query(asset, metric, start = start, end = end, interval = interval)
            df_merged_metric = pd.merge(df_merged_metric, df, how='outer', on=['timestamp', 'asset'])  ### do we loose obs with this merge?
        
        df_merged = pd.concat([df_merged, df_merged_metric], axis=0)

    df_merged["day"] = pd.to_datetime(df_merged.timestamp).dt.date
    df_merged["year"] = pd.to_datetime(df_merged.timestamp).dt.year
    # keep first observation per asset per year
    df_year = df_merged.groupby(["asset", "year"]).last().reset_index()

    return df_merged, df_year

## 2. Select and downlaod data (for different assets, different variables)

In [22]:
# example singe messari query
asset = 'solana'
metric = 'act.addr.cnt'  ##  'mcap.circ', mcap.dom  ## circulating market supply
df1 = messari_query(asset, metric)


solana, act.addr.cnt: Error Not Found
Metric 'act.addr.cnt' for solana is ignored!


In [23]:
#exampe multi messari query
assets = ["bitcoin", "ethereum", "cardano", "polkadot", "solana"] # no data on addresses solona --> error
metrics = ["mcap.circ", "act.addr.cnt", "mcap.dom", "addr.bal.10.cnt"] # circulating_mar
df, df_year = multi_messari_query(assets=assets,metrics=metrics)
df

solana, act.addr.cnt: Error Not Found
Metric 'act.addr.cnt' for solana is ignored!
solana, addr.bal.10.cnt: Error Not Found
Metric 'addr.bal.10.cnt' for solana is ignored!


,timestamp,asset,circulating_marketcap,active_addresses,marketcap_dominance,addresses,day,year
0,2020-01-06 00:00:00+00:00,bitcoin,1.472557e+11,581504.0,69.994370,13943907.0,2020-01-06,2020
1,2020-01-13 00:00:00+00:00,bitcoin,1.606576e+11,622774.0,68.389125,14185190.0,2020-01-13,2020
2,2020-01-20 00:00:00+00:00,bitcoin,1.538085e+11,544915.0,67.975179,14131080.0,2020-01-20,2020
3,2020-01-27 00:00:00+00:00,bitcoin,1.703184e+11,706042.0,67.367427,14341719.0,2020-01-27,2020
4,2020-02-03 00:00:00+00:00,bitcoin,1.835028e+11,695691.0,65.624116,14954464.0,2020-02-03,2020
...,...,...,...,...,...,...,...,...
153,2023-03-13 00:00:00+00:00,solana,8.151069e+09,NaN,0.703298,NaN,2023-03-13,2023
154,2023-03-20 00:00:00+00:00,solana,7.841841e+09,NaN,0.674893,NaN,2023-03-20,2023
155,2023-03-27 00:00:00+00:00,solana,8.131239e+09,NaN,0.675096,NaN,2023-03-27,2023
156,2023-04-03 00:00:00+00:00,solana,7.780007e+09,NaN,0.651450,NaN,2023-04-03,2023


In [24]:
df_year

,asset,year,timestamp,circulating_marketcap,active_addresses,marketcap_dominance,addresses,day
0,bitcoin,2020,2020-12-28 00:00:00+00:00,6.240005e+11,1184688.0,71.262554,20699759.0,2020-12-28
1,bitcoin,2021,2021-12-27 00:00:00+00:00,9.029891e+11,734444.0,39.181441,25836091.0,2021-12-27
2,bitcoin,2022,2022-12-26 00:00:00+00:00,3.182539e+11,719716.0,39.443360,24085339.0,2022-12-26
3,bitcoin,2023,2023-04-10 00:00:00+00:00,5.483222e+11,947327.0,45.521603,27750451.0,2023-04-10
4,cardano,2020,2020-12-28 00:00:00+00:00,5.988532e+09,30733.0,0.683907,446609.0,2020-12-28
5,cardano,2021,2021-12-27 00:00:00+00:00,4.618536e+10,131365.0,2.004021,2299905.0,2021-12-27
6,cardano,2022,2022-12-26 00:00:00+00:00,8.475789e+09,64820.0,1.050462,2473810.0,2022-12-26
7,cardano,2023,2023-04-10 00:00:00+00:00,1.354891e+10,51215.0,1.124828,2731254.0,2023-04-10
8,ethereum,2020,2020-12-28 00:00:00+00:00,1.023436e+11,584367.0,11.687923,10661396.0,2020-12-28
9,ethereum,2021,2021-12-27 00:00:00+00:00,4.481076e+11,551999.0,19.443758,34287615.0,2021-12-27


## 3. Visualizations

In [25]:
fig = px.line(df, x="timestamp", y=["active_addresses"], color="asset").update_layout(height=500)   #, facet_row="subset"
fig.show()

In [26]:
fig = px.line(df, x="timestamp", y=["addresses"], color="asset").update_layout(height=500)   #, facet_row="subset"
fig.show()

In [27]:
fig = px.line(df, x="timestamp", y=["marketcap_dominance"], color="asset").update_layout(height=500)   #, facet_row="subset"
fig.show()

In [28]:
# filter out Bitcoin rows
#df_year_without_btc = df_year[df_year['asset'] != 'bitcoin']
#df_plot = df_year_without_btc

df_plot = df_year

# Get unique years in df_year
years = df_year['year'].unique()

# Loop through years and create a pie chart for each one
for year in years:
    # Filter data for current year
    df_yearly = df_plot[df_plot['year'] == year]
    
    # Create pie chart
    fig = px.pie(df_yearly, values='circulating_marketcap', names='asset')
    fig.update_layout(height=500, width=500, title_text=f"Circ. Market Cap({year})")
    fig.update_traces(textinfo='value', textfont_size=12, textposition='inside')
    fig.show()

In [29]:
df_plot = df_year

# Get unique years in df_year
years = df_year['year'].unique()

# Loop through years and create a pie chart for each one
for year in years:
    # Filter data for current year
    df_yearly = df_plot[df_plot['year'] == year]
    
    # Create pie chart
    fig = px.pie(df_yearly, values='addresses', names='asset')
    fig.update_traces(textinfo='value', textfont_size=12, textposition='inside')

    fig.update_layout(height=500, width=500, title_text=f"Adresses (min 10$) ({year})")
    fig.show()